In [ ]:
!pip install ultralytics
!pip install roboflow

In [1]:
from ultralytics import YOLO
from roboflow import Roboflow
import metaflow


/Users/josetoledo/Desktop/yolov8/yoloenv/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load a model
model = YOLO("yolov8n.yaml")  # build a new model from scratch




                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.Conv                  [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.Conv                  [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.C2f                   [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.Conv                  [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.C2f                   [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.Conv                  [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.C2f                   [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics.nn.modules.Conv                  [128

importar dataset

In [3]:
rf = Roboflow(api_key="KOrKgmkGRHqzxP6SuKTp")
project = rf.workspace("asdasd-boz3q").project("glass-xrghl")
dataset = project.version(3).download("yolov8",location='datasets')


loading Roboflow workspace...
loading Roboflow project...


KeyboardInterrupt: 

In [18]:
# Load a model
#model = YOLO('yolov8n.yaml')  # build a new model from scratch
model = YOLO('yolov8n.pt')  # load a pretrained model (recommended for training)

100%|██████████| 6.24M/6.24M [00:01<00:00, 3.47MB/s]



In [5]:
# Use the model
results = model.train(data="Glass-3/data.yaml", epochs=3, batch=3, imgsz=640, cache='ram')  # train the model salida best.pt



yolo/engine/trainer: task=detect, mode=train, model=None, data=Glass-3/data.yaml, epochs=3, patience=50, batch=3, imgsz=640, save=True, cache=ram, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=False, optimizer=SGD, verbose=False, seed=0, deterministic=True, single_cls=False, image_weights=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, hide_labels=False, hide_conf=False, vid_stride=1, line_thickness=3, visualize=False, augment=False, agnostic_nms=False, retina_masks=False, format=torchscript, keras=False, optimize=False, int8=False, dynamic=False, simplify=False, opset=17, workspace=4, nms=False, lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1

KeyboardInterrupt: 

In [ ]:
results = model.val()  # evaluate model performance on the validation set


In [ ]:

success = YOLO('run/best.pt').export(format='onnx')  # export a model to ONNX format

In [ ]:
results = model('https://ultralytics.com/images/bus.jpg')  # predict on an image

In [10]:
from metaflow import FlowSpec, step
import os

In [11]:


class HelloFlow(FlowSpec):
    """
    A flow where Metaflow prints 'Hi'.

    Run this flow to validate that Metaflow is installed correctly.

    """

    @step
    def start(self):
        """
        This is the 'start' step. All flows must have a step named 'start' that
        is the first step in the flow.

        """

        
        print("HelloFlow is starting.")
        self.next(self.hello)

    @step
    def hello(self):
        """
        A step for metaflow to introduce itself.

        """
        print("Metaflow says: Hi!")
        self.next(self.end)

    @step
    def end(self):
        """
        This is the 'end' step. All flows must have an 'end' step, which is the
        last step in the flow.

        """
        print("HelloFlow is all done.")




In [12]:
HelloFlow()

TypeError: <module '__main__'> is a built-in module

In [13]:
from metaflow import S3,IncludeFile
import json

# Change this URI to that of an S3 bucket you want to write to.
S3_URI = 's3://arn:aws:s3:us-east-1:220318441503:accesspoint/accespoint1'

# Relative, local paths that mirror the structure of the S3 bucket.

data = IncludeFile('peso', default='./runs/detect/train/weights/best.pt')


with S3(s3root=S3_URI) as s3:
    res = json.dumps({'data': data})
    s3.put('data',data)

MetaflowS3InvalidObject: Object corresponding to the key 'peso' is not a string or a bytes object.

In [2]:

with S3(s3root=S3_URI) as s3:
    s3.put_files('runs/detect/train/weights/best.pt')

AttributeError: 'str' object has no attribute 'key'

In [ ]:
import zipfile
import os
from metaflow import S3

def _download_from_s3(file):
    with S3(s3root = self.S3_URI) as s3:
        result = s3.get(file)
        with zipfile.ZipFile(result.path, 'r') as zip_ref:
            zip_ref.extractall(
                os.path.join(DATA_ROOT, file.split('.zip')[0])
            )
            
# EXAMPLE USES 
# _download_from_s3('subsample.zip')
# _download_from_s3('subsample-annotations.zip')

In [7]:
if __name__ == "__main__":
    HelloFlow()

TypeError: <module '__main__'> is a built-in module

In [ ]:
from metaflow import FlowSpec, step, S3
import json

class S3DemoFlow(FlowSpec):

    @step
    def start(self):
        with S3(run=self) as s3:
            message = json.dumps({'message': 'hello world!'})
            url = s3.put('example_object', message)
            print("Message saved at", url)
        self.next(self.end)

    @step
    def end(self):
        with S3(run=self) as s3:
            s3obj = s3.get('example_object')
            print("Object found at", s3obj.url)
            print("Message:", json.loads(s3obj.text))

if __name__ == '__main__':
    S3DemoFlow()